In [2]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
comments = pd.read_csv('../data/comments_lang.csv')
comments_en = comments[comments.lang == 'en']
vect = CountVectorizer(ngram_range = (1,1), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 200,
                       min_df = 2, max_df = 0.95).fit(comments_en.Review)

In [5]:
pw = list(vect.vocabulary_.keys())

In [6]:
def check_spelling(text):
    import enchant
    c = enchant.Dict("en_UK")
    if not c.check(text):
        suggestions = list(set(c.suggest(text)).intersection(set(pw)))
        if len(suggestions)>0:
            res = suggestions[0]
        else:
            res = text
    else:
        res = text
    return res

In [7]:
check_spelling('subpar')

'super'

In [8]:
labeled1 = pd.read_excel('manual_labels.xlsx')
labeled2 = pd.read_excel('manual_labels2.xlsx')
labeled3 = pd.read_excel('manual_labels3.xlsx')
labeled4 = pd.read_excel('manual_labels4.xlsx')

In [9]:
labeled4.label.value_counts()

4    574
2    204
5     93
0     77
3     42
1     10
Name: label, dtype: int64

In [10]:
labeled2.shape

(1000, 5)

In [11]:
labeled1.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [12]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    res = ''
    for t in tokens:
        res += wnl.lemmatize(t)+' '
    return res
def get_tokens(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    return tokens

In [13]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\!',' exclamation_point ', text)
    tokens = word_tokenize(deacc)
    tags = nltk.pos_tag(tokens)
    processed = []
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(word,wn_tag))
        else:
            processed.append(wnl.lemmatize(check_spelling(word)))
    res = ' '.join(processed)
    return res

In [14]:
labeled = pd.concat([labeled1, labeled2, labeled3, labeled4],axis = 0)

In [15]:
labeled.loc[:,'cleaned'] = labeled.Review.apply(clean_comment)

In [16]:
labeled['tokens'] = labeled.Review.apply(get_tokens)

In [17]:
labeled_long = labeled[labeled.tokens.apply(len)>6]
labeled_neg = labeled[labeled.label!=4]

In [18]:
comments_en['cleaned'] = comments_en.Review.apply(clean_comment)

/home/lsm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [815]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LassoCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [816]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,3), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 600,
                       min_df = 2, max_df = 0.95).fit(comments_en.cleaned)
feats = vect.transform(labeled.cleaned).toarray()
labels = labeled.label.as_matrix()

In [817]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    
    class_names_b = ['neg', 'pos']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [818]:
feats.shape

(4000, 600)

In [819]:
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [820]:
m = OneVsRestClassifier(DummyClassifier()).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(m, feats,labels))))

train scores

                  precision    recall  f1-score   support

         unknown       0.11      0.03      0.05       223
           Crash       0.00      0.00      0.00        27
Balance problems       0.21      0.08      0.11       652
 Synchronization       0.00      0.00      0.00       103
        Positive       0.60      0.54      0.57      1925
             Bug       0.10      0.40      0.15       270

     avg / total       0.42      0.38      0.38      3200

test scores

                  precision    recall  f1-score   support

         unknown       0.20      0.05      0.08        57
           Crash       0.00      0.00      0.00         8
Balance problems       0.17      0.06      0.09       148
 Synchronization       0.00      0.00      0.00        26
        Positive       0.67      0.55      0.60       510
             Bug       0.08      0.45      0.13        51

     avg / total       0.48      0.40      0.42       800

cross_val_score: 0.3855


In [835]:
m = RidgeClassifier(class_weight = 'balanced', alpha = 25).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(m, feats,labels))))

train scores

                  precision    recall  f1-score   support

         unknown       0.31      0.87      0.46       223
           Crash       0.26      1.00      0.42        27
Balance problems       0.86      0.67      0.75       652
 Synchronization       0.69      0.92      0.79       103
        Positive       0.94      0.76      0.84      1925
             Bug       0.69      0.73      0.71       270

     avg / total       0.84      0.75      0.78      3200

test scores

                  precision    recall  f1-score   support

         unknown       0.27      0.79      0.41        57
           Crash       0.04      0.12      0.06         8
Balance problems       0.77      0.60      0.68       148
 Synchronization       0.59      0.50      0.54        26
        Positive       0.93      0.76      0.84       510
             Bug       0.49      0.59      0.54        51

     avg / total       0.81      0.70      0.74       800

cross_val_score: 0.6845


In [836]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model = m):
    cleaned_text = clean_comment(input_text)
    feats = vect.transform([cleaned_text])
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats.toarray())
    #print(class_names[prediction[0]])
    return prediction[0]
def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_classifier))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    return y_true,y_pred
y_true,y_pred = val_score(m)

                  precision    recall  f1-score   support

           Other       0.77      1.00      0.87        10
           Crash       0.67      0.60      0.63        10
Balance problems       0.50      0.80      0.62        10
 Synchronization       0.78      0.70      0.74        10
        Positive       0.83      0.50      0.62        10
             Bug       0.71      0.50      0.59        10

     avg / total       0.71      0.68      0.68        60



In [837]:
eval_classifier('this dum gaem crashes every time i launch it')

1

In [838]:
eval_classifier('the new up date made game sooo unfair')

2

In [839]:
eval_classifier('the game glitched and half of my guns now lost!')

3

In [840]:
eval_classifier('Cool!')

4

In [841]:
eval_classifier('This game is haard to control')

5